<a href="https://colab.research.google.com/github/jxxngho/HG_MLDL/blob/main/05_2_Cross_validation_and_Grid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검증 세트

In [1]:
# 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 힘들다.
# 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는 것!
# 이를 검증 세트라고 한다.

# 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가
# 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다
# 그다음 이 매개변수를 이용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련
# 그리고 마지막에 테스트 세트에서 최종 점수를 평가한다!!!

In [2]:
# 데이터 불러오기
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
# 훈련 세트와 테스트 세트로 나누기
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data,target, test_size=0.2, random_state=42)

print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [6]:
# 이 다음 train_input, train_target을 다시 넣어 훈련 세트와 검증 세트를 만든다.

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [9]:
# 단순히 train_test_split 함수를 두 번 적용해서 훈련 세트와 검증 세트로 나눈 것 뿐!
print(sub_input.shape, val_input.shape)

# 원래 5197개였던 훈련 세트가 4157개로 줄고, 검증세트는 1040개가 됨

(4157, 3) (1040, 3)


In [10]:
# 모델 학습 및 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)

# 훈련세트와 검증세트로 점수 평가
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

# 훈련 세트에 과대적합 되어있다. 매개변수를 바꾸어 더 좋은 모델을 찾자!

0.9971133028626413
0.864423076923077


# 교차검증

In [11]:
# 검증 세트를 만드느라 훈련 세트가 줄었다 ㅠ_ㅠ 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어짐.
# 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복 -> 이 점수를 평균하여 최종 검증 점수를 얻는다.
# ex) k-폴드 교차 검증
# 보통 5폴드나 10폴트 교차검증 많이 사용

# 사이킷런에서는 cross_validate() 교차 검증 함수 제공
# 평가할 모델 객체를 첫 번째 매개변수로 전달한 후
# 직접 검증 세트를  떼어 내지 않고 훈련 세트 전체를 cross_validate()함수에 전달

from sklearn.model_selection import cross_validate
# 검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수라고 여겨진다..??
scores = cross_validate(dt, train_input, train_target)
# cross_validate 기본값은 5
print(scores)

{'fit_time': array([0.00900459, 0.00598812, 0.00643086, 0.00610876, 0.00578809]), 'score_time': array([0.00123906, 0.00080466, 0.00081491, 0.00078464, 0.00075436]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [12]:
# 이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환
#처음 두 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미

In [13]:
# 교차 검증의 최종 점수는 평균낼 수 있음

import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [15]:
# cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다!
# train_test_split()함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없었다.
# 만약 섞을 필요가 있다면 분할기를 지정해줘야한다.

from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [16]:
# 만약 훈련 세트를 섞은 후 10-폴드 교차 검증 수행하려면?

splitter = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores = cross_validate(dt,train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [18]:
# 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아보자~!
# 이 때 테스트 세트를 사용하지 않고 교차 검증을 통해서 좋은 모델을 고르면 된다.

# 하이퍼파라미터 튜닝

In [23]:
# 하이퍼파라미터란 사용자 지정 파라미터이다.

# 하이퍼파라미터 튜닝하는 작업
# 먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델 훈련
# 그다음 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꾼다.

# 그리드 서치 - 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행
# 별도로 cross_validate()함수를 호출할 필요가 없다.

from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001,0.0002,0.0003,0.0004,0.0005]}
# min_impurity_decrease : 정보이득, 불순도 차이가 클수록 좋음 -> 분할 잘한 것
# ex) 0.1값이면 ,, 0.1도 안되면 분할하지마! 가지치기 하나로 사용할 수 있음.


gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=1)

gs.fit(train_input,train_target)
# 교차 검증에서 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 한다.
# 사이킷런의 그리드 서치는 훈련이 끝나면 25개 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로
# 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.
# 이 모델은 gs 객체의 best_estimtor_ 속성에 저장되어 있다.
# 이 모델을 일반 결정 트리처럼 똑같이 사용 가능.

dt = gs.best_estimator_
print(dt.score(train_input,train_target))
# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있다.
print(gs.best_params_)

0.9615162593804117
{'min_impurity_decrease': 0.0001}


In [24]:
# 5번의 교차 검증으로 얻은 점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [21]:
# 넘파이 argmax()함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.
# 그 다음 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다.

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [25]:
# 이 과정을 정리해보자!!

# 1. 먼저 탐색할 매개변수를 지정한다.
# 2. 그 다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다.
#    이 조합은 그리드 서치 객체에 저장된다.
# 3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련한다.
#    이 모델도 그리드 서치 객체에 저장된다.

# 결정 트리에서 min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정한다.
# max_depth로 트리의 깊이를 제한하고 min_samples_split으로 노드를 나누기 위한 최소 샘플 수도 골라보자

params = {'min_impurity_decrease' : np.arange(0.0001,0.001,0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2,100,10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

# 이해못한 부분 다시 체크하고 진도나가기!

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [29]:
# 더 복잡한 모델을 만들어보자!

# min_impurity_decrease : 노드를 분할하기 위한 불순도 감소 최소량
# max_depth : 트리 깊이 제한
# min_samples_split : 노드를 나누기 위한 최소 샘플 수

params = {'min_impurity_decrease' : np.arange(0.0001,0.001,0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2,100,10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
gs.fit(train_input,train_target)

# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [30]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# 랜덤 서치

In [ ]:
# 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다
# 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다
# 이럴 때 랜덤 서치를 사용하면 된다!!

# 랜덤 서치는 매개변수 값의 목록을 전달하는 것이 아니라
# "매개변수를 샘플링할 수 있는 확률 분포 객체를 전달"

In [31]:
from scipy.stats import uniform, randint
# stats 서브 패키지에 있는 uniform과 randint 클래스는 모든 주어진 범위에서
# 고르게 값을 뽑는다. --> '균등 분포에서 샘플링한다'
# 0~10 사이의 범위를 갖는 randint 객체를 만들고 10개의 숫자를 샘플링해보자

In [34]:
rgen = randint(0,10)
rgen.rvs(10)

array([7, 3, 1, 4, 8, 4, 0, 4, 1, 6])

In [37]:
# 1000개 샘플링해서 각 숫자의 개수를 세어보기
print(np.unique(rgen.rvs(1000),return_counts=True))

# 난수 발생기랑 유사하게 생각하면 된다.


(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([100, 101, 111, 101, 103,  96, 106,  87, 101,  94]))


In [53]:
# min_samples_lear 매개변수를 탐색 대상에 추가
# 이 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수
# 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않음.

params = {'min_impurity_decrease' : np.arange(0.0001,0.001),
          'max_depth' : range(20,50),
          'min_samples_split' : range(2,25),
          'min_samples_leaf' : randint(1,25)
          }

In [54]:
# 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params, n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

# 위 params에 정의된 매개변수 범위에서 총 100번을 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾는다.
# 앞서 그리드 서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영억으로 탐색 가능!

# 최적의 매개변수 조합
print(gs.best_params_)

# 최고의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

# 최적 모델은 이미 전체 훈련 세트로 훈련되어 best_estimator_ 속성에 저장되어 있음
# 이 모델을 최종 모델로 결정하고 테스트 세트의 성능을 확인해보자

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 31, 'min_impurity_decrease': 0.0001, 'min_samples_leaf': 23, 'min_samples_split': 21}
0.8683852817057822
0.8576923076923076
